<a href="https://colab.research.google.com/github/vcngeno/Isolation_Forest_Unsupervised_Learning/blob/main/Isolation_Forest_Unsupervised_Learning_Vanessa_Ngeno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Feature Engineering

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import category_encoders as ce

In [ ]:
#Loading the dataset and preview
df = pd.read_csv('/content/OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025 5.csv')
df.head()

,Change_Type,Physician_Profile_ID,Physician_NPI,Physician_First_Name,Physician_Middle_Name,Physician_Last_Name,Physician_Name_Suffix,Recipient_Primary_Business_Street_Address_Line1,Recipient_Primary_Business_Street_Address_Line2,Recipient_City,...,Value_of_Interest,Terms_of_Interest,Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country,Dispute_Status_for_Publication,Interest_Held_by_Physician_or_an_Immediate_Family_Member,Payment_Publication_Date
0,UNCHANGED,329721,1.275588e+09,Frank,P,Cammisa,JR,523 E 72ND ST,NaN,New York,...,59000.00,COMMON STOCK,"4WEB, INC.",1.000000e+11,"4WEB, INC.",TX,United States,No,Physician Covered Recipient,1/30/25
1,UNCHANGED,4213213,1.952690e+09,Edward,NaN,Shields,NaN,1400 MERCY DR,NaN,Muskegon,...,3089.19,Stock,Catalyst OrthoScience,1.000000e+11,Catalyst OrthoScience,FL,United States,No,Physician Covered Recipient,1/30/25
2,UNCHANGED,170314,1.750495e+09,ERIK,NaN,THYSSEN,NaN,11525 Olde Cabin Road,NaN,ST LOUIS,...,5709.14,Common Stock,"GastroGPO, LLC",1.000010e+11,"GastroGPO, LLC",OH,United States,No,Physician Covered Recipient,1/30/25
3,UNCHANGED,151386,1.336162e+09,Cindy,NaN,Tobin,NaN,40 Crossways Park Dr.,Suite 101,Woodbury,...,1.00,Physician holds an indirect ownership interest...,"Unified Women's Healthcare, LP",1.000000e+11,"Unified Women's Healthcare, LP",FL,United States,No,Physician Covered Recipient,1/30/25
4,UNCHANGED,130749,1.871711e+09,Nikhil,NaN,Thakur,NaN,575 Turnpike St,Suite 11,North Andover,...,7017.40,Restricted Stock Award per consultant agreement.,"Expanding Innovations, Inc.",1.000010e+11,"Expanding Innovations, Inc.",CA,United States,No,Physician Covered Recipient,1/30/25


In [ ]:
# Define columns
numerical_cols = ['Total_Amount_Invested_USDollars', 'Value_of_Interest', 'Program_Year']
categorical_cols = ['Physician_Specialty', 'Recipient_State', 'Change_Type',
                    'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name']

**Explanation:**

I selected the above features out of logic because they are not a unique identifier. The rest are.

In [ ]:
#Select features
X = df[numerical_cols + categorical_cols]

In [ ]:
#Checking data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4013 entries, 0 to 4012
Data columns (total 30 columns):
 #   Column                                                            Non-Null Count  Dtype  
---  ------                                                            --------------  -----  
 0   Change_Type                                                       4013 non-null   object 
 1   Physician_Profile_ID                                              4013 non-null   int64  
 2   Physician_NPI                                                     4010 non-null   float64
 3   Physician_First_Name                                              4013 non-null   object 
 4   Physician_Middle_Name                                             1211 non-null   object 
 5   Physician_Last_Name                                               4013 non-null   object 
 6   Physician_Name_Suffix                                             108 non-null    object 
 7   Recipient_Primary_Business_Street

In [ ]:
#Checking for missing data
print(df.isnull().sum())

Change_Type                                                            0
Physician_Profile_ID                                                   0
Physician_NPI                                                          3
Physician_First_Name                                                   0
Physician_Middle_Name                                               2802
Physician_Last_Name                                                    0
Physician_Name_Suffix                                               3905
Recipient_Primary_Business_Street_Address_Line1                        0
Recipient_Primary_Business_Street_Address_Line2                     2137
Recipient_City                                                         0
Recipient_State                                                        0
Recipient_Zip_Code                                                     0
Recipient_Country                                                      0
Recipient_Province                                 

In [ ]:
# Split into train/test
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

In [ ]:
# Imputation on both numeric and categorical variables because both contain missing values
X_train[numerical_cols] = X_train[numerical_cols].fillna(X_train[numerical_cols].median())
X_test[numerical_cols] = X_test[numerical_cols].fillna(X_train[numerical_cols].median())  # Use train median
X_train[categorical_cols] = X_train[categorical_cols].fillna(X_train[categorical_cols].mode().iloc[0])
X_test[categorical_cols] = X_test[categorical_cols].fillna(X_train[categorical_cols].mode().iloc[0])

In [ ]:
# One-hot encoding for the categorical variables
encoder = ce.OneHotEncoder(cols=categorical_cols, use_cat_names=True, handle_unknown='indicator')
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)  # Use same encoder

In [ ]:
# Standardizing
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_encoded), columns=X_train_encoded.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_encoded), columns=X_test_encoded.columns)

In [ ]:
# Verify no NaNs
print(X_train_scaled.isnull().sum().sum())  # Should be 0

0


#Modeling

In [ ]:
from pyod.models.iforest import IForest

# Initialize Isolation Forest
model = IForest(contamination=0.1, random_state=42)  # 10% expected anomalies

# Fit the model on training data
model.fit(X_train_scaled)

IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=1, random_state=42,
    verbose=0)

In [ ]:
# Get anomaly scores and predictions
train_scores = model.decision_function(X_train_scaled)  # Anomaly scores
test_scores = model.decision_function(X_test_scaled)
train_preds = model.predict(X_train_scaled)  # 1 = anomaly, 0 = inlier
test_preds = model.predict(X_test_scaled)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(


In [ ]:
# Top anomalies (lowest scores = most anomalous)
top_anomalies = pd.DataFrame({'Index': range(len(test_scores)), 'Score': test_scores})
top_anomalies = top_anomalies.sort_values(by='Score').head(10)
print("Top 10 anomalies based on anomaly scores:")
print(top_anomalies)
print(f"Number of anomalies flagged: {sum(test_preds == 1)}")

Top 10 anomalies based on anomaly scores:
     Index     Score
385    385 -0.029289
710    710 -0.029082
152    152 -0.029027
762    762 -0.027503
55      55 -0.027373
36      36 -0.027346
197    197 -0.027303
667    667 -0.027196
373    373 -0.026616
251    251 -0.026217
Number of anomalies flagged: 86


#Explain the Method

The Isolation Forest, configured with contamination=0.1 and random_state=42 (Page 2), builds random trees to isolate anomalies in the dataset. It splits features like Total_Amount_Invested_USDollars and Physician_Specialty randomly, flagging points with shorter path lengths (lower scores, e.g., -0.029289) as anomalies, targeting 10% of the data as outliers.

#Insights Why They Are Identified as Anomalous

The top 10 anomalies (e.g., Index 385, Score -0.029289; Index 710, Score -0.029082) are likely flagged due to extreme values in Total_Amount_Invested_USDollars or Value_of_Interest (e.g., millions vs. typical thousands), rare Physician_Specialty (e.g., niche fields), uncommon Recipient_State (e.g., low-frequency states), or unique Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name. With 86 anomalies flagged, these records deviate significantly from the majority, making them easy to isolate in the feature space.